In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/pggan/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 7000
start_epoch = 0
train_batch = 12
test_batch = 200
lr = 0.1
schedule = [200, 1500, 3000, 4500, 6000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b1/to_style1/cutmix2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = 'fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.1

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(target_dir, '100_shot_pggan_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        source_inputs, source_targets = source_set[batch_idx]
        source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()

        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        # teacher model output, prediction

#         teacher_outputs = nn.Softmax()(teacher_model(inputs))
#         pred = teacher_outputs.argmax(dim=-1)
        lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        target_a = targets
        target_b = targets[rand_index]

        outputs = student_model(inputs)
        loss_main = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(source_train_loader, train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 7000] LR: 0.100000
1/17 Data:3.461 | Batch:8.735 | Total:0:00:06 | ETA:0:01:45 | Loss:0.10133863240480423 | top1:91.66667175292969
2/17 Data:0.003 | Batch:0.870 | Total:0:00:07 | ETA:0:00:56 | Loss:1.06427538767457 | top1:79.16667175292969
3/17 Data:0.001 | Batch:0.799 | Total:0:00:08 | ETA:0:00:39 | Loss:1.2566504900654156 | top1:75.0
4/17 Data:0.003 | Batch:0.586 | Total:0:00:08 | ETA:0:00:29 | Loss:2.381602557376027 | top1:62.5
5/17 Data:0.003 | Batch:0.487 | Total:0:00:09 | ETA:0:00:23 | Loss:2.74617607742548 | top1:60.000003814697266
6/17 Data:0.002 | Batch:0.806 | Total:0:00:10 | ETA:0:00:19 | Loss:2.9671921941141286 | top1:61.11111068725586
7/17 Data:0.015 | Batch:0.709 | Total:0:00:10 | ETA:0:00:16 | Loss:3.1107068583369255 | top1:60.71428680419922
8/17 Data:0.004 | Batch:0.637 | Total:0:00:11 | ETA:0:00:13 | Loss:3.1968058506026864 | top1:63.54166793823242
9/17 Data:0.003 | Batch:0.959 | Total:0:00:12 | ETA:0:00:12 | Loss:3.2366328115264573 | top1:67.5925903320312

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


39/39 Data:0.002 | Batch:0.372 | Total:0:00:30 | ETA:0:00:00 | Loss:3.6843647406651425 | top1:50.0128173828125
161/161 Data:0.002 | Batch:1.067 | Total:0:00:56 | ETA:0:00:00 | Loss:2.9627192904273296 | top1:98.5077896118164

Epoch: [2 | 7000] LR: 0.106000
1/17 Data:3.291 | Batch:3.878 | Total:0:00:01 | ETA:0:00:22 | Loss:2.9545514583587646 | top1:100.0
2/17 Data:0.004 | Batch:0.473 | Total:0:00:01 | ETA:0:00:14 | Loss:3.0598554611206055 | top1:91.66667175292969
3/17 Data:0.002 | Batch:0.751 | Total:0:00:02 | ETA:0:00:12 | Loss:3.1244277159372964 | top1:83.33333587646484
4/17 Data:0.004 | Batch:0.980 | Total:0:00:03 | ETA:0:00:12 | Loss:3.135521173477173 | top1:83.33333587646484
5/17 Data:0.002 | Batch:0.912 | Total:0:00:04 | ETA:0:00:11 | Loss:3.1465101718902586 | top1:80.00000762939453
6/17 Data:0.001 | Batch:0.825 | Total:0:00:05 | ETA:0:00:10 | Loss:3.0983728965123496 | top1:83.33333587646484
7/17 Data:0.002 | Batch:0.698 | Total:0:00:05 | ETA:0:00:09 | Loss:3.101950236729213 | top1

9/17 Data:0.003 | Batch:0.503 | Total:0:00:04 | ETA:0:00:05 | Loss:1.2036879592471652 | top1:83.33333587646484
10/17 Data:0.003 | Batch:0.302 | Total:0:00:05 | ETA:0:00:04 | Loss:1.1739046633243562 | top1:84.16667175292969
11/17 Data:0.006 | Batch:0.399 | Total:0:00:05 | ETA:0:00:04 | Loss:1.1535539410331033 | top1:84.09091186523438
12/17 Data:0.002 | Batch:0.442 | Total:0:00:06 | ETA:0:00:03 | Loss:1.1356883744398754 | top1:84.72222137451172
13/17 Data:0.005 | Batch:0.480 | Total:0:00:06 | ETA:0:00:02 | Loss:1.1134134301772485 | top1:85.25640869140625
14/17 Data:0.002 | Batch:0.474 | Total:0:00:07 | ETA:0:00:02 | Loss:1.1098109355994634 | top1:84.52381134033203
15/17 Data:0.001 | Batch:0.549 | Total:0:00:07 | ETA:0:00:01 | Loss:1.1168709635734557 | top1:83.33333587646484
16/17 Data:0.002 | Batch:0.470 | Total:0:00:08 | ETA:0:00:01 | Loss:1.1054642014205456 | top1:83.85417175292969

Epoch: [7 | 7000] LR: 0.136000
1/17 Data:1.733 | Batch:2.068 | Total:0:00:00 | ETA:0:00:15 | Loss:0.8204

3/17 Data:0.001 | Batch:0.299 | Total:0:00:01 | ETA:0:00:09 | Loss:1.5579733848571777 | top1:88.8888931274414
4/17 Data:0.001 | Batch:0.448 | Total:0:00:02 | ETA:0:00:08 | Loss:1.5603946149349213 | top1:89.58333587646484
5/17 Data:0.004 | Batch:0.356 | Total:0:00:02 | ETA:0:00:07 | Loss:1.5591175079345703 | top1:86.66667175292969
6/17 Data:0.001 | Batch:0.337 | Total:0:00:02 | ETA:0:00:06 | Loss:1.5513798197110493 | top1:88.8888931274414
7/17 Data:0.002 | Batch:0.491 | Total:0:00:03 | ETA:0:00:05 | Loss:1.5652803352900915 | top1:89.28571319580078
8/17 Data:0.003 | Batch:0.583 | Total:0:00:04 | ETA:0:00:05 | Loss:1.565277636051178 | top1:87.5
9/17 Data:0.001 | Batch:0.511 | Total:0:00:04 | ETA:0:00:05 | Loss:1.5507841772503324 | top1:87.96296691894531
10/17 Data:0.001 | Batch:0.353 | Total:0:00:04 | ETA:0:00:04 | Loss:1.5398143768310546 | top1:88.33333587646484
11/17 Data:0.002 | Batch:0.393 | Total:0:00:05 | ETA:0:00:03 | Loss:1.5142286582426592 | top1:88.63636779785156
12/17 Data:0.00

...

12/17 Data:0.036 | Batch:2.406 | Total:0:00:06 | ETA:0:00:03 | Loss:0.41129428644975025 | top1:99.30555725097656
13/17 Data:0.001 | Batch:4.031 | Total:0:00:10 | ETA:0:00:04 | Loss:0.40347875310824466 | top1:99.35897827148438
14/17 Data:0.003 | Batch:3.539 | Total:0:00:14 | ETA:0:00:04 | Loss:0.4104041648762567 | top1:98.80952453613281
15/17 Data:0.001 | Batch:4.718 | Total:0:00:19 | ETA:0:00:04 | Loss:0.4124600430329641 | top1:98.8888931274414
16/17 Data:0.001 | Batch:4.180 | Total:0:00:23 | ETA:0:00:03 | Loss:0.40922017581760883 | top1:98.95833587646484

Epoch: [889 | 7000] LR: 0.038605
1/17 Data:13.286 | Batch:13.635 | Total:0:00:01 | ETA:0:00:22 | Loss:0.3576194941997528 | top1:100.0
2/17 Data:0.026 | Batch:0.592 | Total:0:00:01 | ETA:0:00:15 | Loss:0.3321777433156967 | top1:100.0
3/17 Data:0.002 | Batch:3.683 | Total:0:00:05 | ETA:0:00:27 | Loss:0.4235167404015859 | top1:97.22222137451172
4/17 Data:0.026 | Batch:3.732 | Total:0:00:09 | ETA:0:00:31 | Loss:0.5287812873721123 | top1:

7/17 Data:0.001 | Batch:5.101 | Total:0:00:36 | ETA:0:00:52 | Loss:0.39432424306869507 | top1:100.0
8/17 Data:0.001 | Batch:4.211 | Total:0:00:40 | ETA:0:00:46 | Loss:0.3826036751270294 | top1:100.0
9/17 Data:0.001 | Batch:5.122 | Total:0:00:45 | ETA:0:00:41 | Loss:0.3734375503328111 | top1:100.0
10/17 Data:0.001 | Batch:3.286 | Total:0:00:48 | ETA:0:00:35 | Loss:0.3665541023015976 | top1:100.00000762939453
11/17 Data:0.004 | Batch:4.512 | Total:0:00:53 | ETA:0:00:26 | Loss:0.3683391403068196 | top1:100.0
12/17 Data:0.001 | Batch:3.325 | Total:0:00:56 | ETA:0:00:22 | Loss:0.36886489391326904 | top1:100.0
13/17 Data:0.001 | Batch:3.830 | Total:0:01:00 | ETA:0:00:18 | Loss:0.37034783455041737 | top1:100.0
14/17 Data:0.002 | Batch:3.216 | Total:0:01:03 | ETA:0:00:13 | Loss:0.3694098251206534 | top1:100.0
15/17 Data:0.008 | Batch:4.312 | Total:0:01:07 | ETA:0:00:09 | Loss:0.3767973979314168 | top1:99.44445037841797
16/17 Data:0.001 | Batch:3.548 | Total:0:01:11 | ETA:0:00:05 | Loss:0.39253

3/17 Data:0.001 | Batch:4.236 | Total:0:00:09 | ETA:0:00:46 | Loss:0.4374099373817444 | top1:94.44444274902344
4/17 Data:0.001 | Batch:3.742 | Total:0:00:13 | ETA:0:00:44 | Loss:0.417313814163208 | top1:95.83333587646484
5/17 Data:0.001 | Batch:3.812 | Total:0:00:17 | ETA:0:00:42 | Loss:0.421761566400528 | top1:95.00000762939453
6/17 Data:0.001 | Batch:1.535 | Total:0:00:18 | ETA:0:00:35 | Loss:0.41229001184304553 | top1:95.83333587646484
7/17 Data:0.001 | Batch:3.509 | Total:0:00:22 | ETA:0:00:32 | Loss:0.40243279933929443 | top1:96.42857360839844
8/17 Data:0.001 | Batch:3.396 | Total:0:00:25 | ETA:0:00:29 | Loss:0.40354832634329796 | top1:96.875
9/17 Data:0.001 | Batch:3.197 | Total:0:00:28 | ETA:0:00:26 | Loss:0.39602184626791215 | top1:97.22222137451172
10/17 Data:0.002 | Batch:3.843 | Total:0:00:32 | ETA:0:00:23 | Loss:0.3895357996225357 | top1:97.50000762939453
11/17 Data:0.001 | Batch:4.370 | Total:0:00:37 | ETA:0:00:22 | Loss:0.3798724494197152 | top1:97.7272720336914
12/17 Dat

12/17 Data:0.006 | Batch:3.843 | Total:0:00:37 | ETA:0:00:15 | Loss:0.375491663813591 | top1:99.30555725097656
13/17 Data:0.001 | Batch:4.078 | Total:0:00:41 | ETA:0:00:12 | Loss:0.37083335793935335 | top1:99.35897827148438
14/17 Data:0.007 | Batch:4.439 | Total:0:00:46 | ETA:0:00:09 | Loss:0.3741618607725416 | top1:98.80952453613281
15/17 Data:0.003 | Batch:4.855 | Total:0:00:50 | ETA:0:00:07 | Loss:0.37571552991867063 | top1:98.8888931274414
16/17 Data:0.003 | Batch:4.407 | Total:0:00:55 | ETA:0:00:04 | Loss:0.3687245063483715 | top1:98.95833587646484

Epoch: [903 | 7000] LR: 0.038559
1/17 Data:17.959 | Batch:21.500 | Total:0:00:09 | ETA:0:02:26 | Loss:0.32889774441719055 | top1:100.0
2/17 Data:0.002 | Batch:2.470 | Total:0:00:11 | ETA:0:01:27 | Loss:0.336437463760376 | top1:100.0
3/17 Data:0.001 | Batch:0.761 | Total:0:00:12 | ETA:0:00:58 | Loss:0.3807372550169627 | top1:100.0
4/17 Data:0.013 | Batch:0.370 | Total:0:00:12 | ETA:0:00:42 | Loss:0.40742916613817215 | top1:97.9166717529

5/17 Data:0.001 | Batch:0.424 | Total:0:00:25 | ETA:0:01:02 | Loss:0.4112132161855698 | top1:91.66667175292969
6/17 Data:0.003 | Batch:0.253 | Total:0:00:25 | ETA:0:00:48 | Loss:0.4034229690829913 | top1:93.05555725097656
7/17 Data:0.003 | Batch:0.354 | Total:0:00:26 | ETA:0:00:38 | Loss:0.40587387553283144 | top1:94.04762268066406
8/17 Data:0.002 | Batch:0.661 | Total:0:00:26 | ETA:0:00:31 | Loss:0.37870118394494057 | top1:94.79167175292969
9/17 Data:0.002 | Batch:1.085 | Total:0:00:27 | ETA:0:00:25 | Loss:0.3752103414800432 | top1:95.37036895751953
10/17 Data:0.003 | Batch:3.760 | Total:0:00:31 | ETA:0:00:23 | Loss:0.3783713400363922 | top1:95.83333587646484
11/17 Data:0.001 | Batch:3.380 | Total:0:00:35 | ETA:0:00:15 | Loss:0.38426952199502423 | top1:95.45455169677734
12/17 Data:0.001 | Batch:4.084 | Total:0:00:39 | ETA:0:00:12 | Loss:0.3834891642133395 | top1:95.83333587646484
13/17 Data:0.001 | Batch:4.528 | Total:0:00:43 | ETA:0:00:09 | Loss:0.3760136388815366 | top1:96.153846740